In [ ]:
import torch
import sys
import gc
sys.path.append('..')
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, SMOL_LM_CONFIG

In [ ]:
from dataclasses import replace
cfg = replace(SMOL_LM_CONFIG, max_samples=100, single_process_mode=True)
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [40]:
wrapper.init_trainer()

2024-12-18 03:19:14.526 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:138 - LoRA already loaded
2024-12-18 03:19:14.802 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:185 - Initializing DPOTrainer, with project name: codecontests-llama-3b, run_name: run-390448, logprobs cache: dataset_caches/ultrafeedback/b68b303b/ref_logprobs_cache
2024-12-18 03:19:15.302 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:206 - Loading cached logprobs...


In [ ]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

In [43]:
wrapper.train()

2024-12-18 03:21:05.270 | INFO     | trl_wrapper.trainer_wrapper:train:312 - Starting training.
2024-12-18 03:21:06.197 | INFO     | trl_wrapper.dpo_trainer:evaluation_loop:120 - Generating samples...
2024-12-18 03:21:06.199 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:48 - Generating policy samples, max length: 512...
2024-12-18 03:21:11.488 | INFO     | trl_wrapper.dpo_trainer:generate_from_model_and_ref:66 - Generating reference samples...
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


┌──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┬──────────────────────────────────────────┐
│ prompt                                   │ policy                                   │ ref                                      │ chosen                                   │ rejected                                 │
├──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┤
│ system You are Dolphin, a helpful AI     │ Let's start with the title "Never Let Me │ Let's start with the title "Never Let Me │ and are discouraged from developing      │                                          │
│ assistant. user Let's put your puzzle-   │ Go." The story revolves around a group   │ Go." The story revolves around a group   │ e

KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown
for prompt, chosen, rejected, logprob_diff in []:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))